# SPML HW3 - Defensive Distillation (30)

### Deadline: 1402/2/30

#### Name: 
#### Student No.: 

In this notebook, you are going to attack a defensively distilled model

Please write your code in specified sections and do not change anything else. If you have a question regarding this homework, please ask it on the course page.

Also, it is recommended to use google colab to do this homework. You can connect to your drive using the code below:

## Initializations

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.models import resnet18
import torch.optim as optim

## Defining Teacher and Student Classes

In [ ]:
class Teacher(nn.Module):
  def __init__(self, num_cls, T=1):
    super().__init__()
    self.conv = nn.Sequential(
        *list(resnet18(pretrained=False).children())[:-2])
    
    self.fc = nn.Linear(512, num_cls)
    self.temp = T
  
  def forward(self, x, T=None):
    if T is None:
      T = self.temp
    x = self.conv(x)
    x = torch.flatten(x, start_dim=1)
    logits = self.fc(x)
    output = torch.softmax(logits / T, dim=1)

    return logits, output

class Student(nn.Module):
  def __init__(self, num_cls, T=1):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, num_cls)
    self.temp = T

  def forward(self, x, T=None):
    if T is None:
      T = self.temp
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    logits = self.fc3(x)

    output = torch.softmax(logits / T, dim=1)
    return logits, output

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

###################### Problem 1 (3 points) ####################################
# todo: Define your data loaders for training, testing, and validation         #
################################################################################

# your code goes here

################################ End ###########################################

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Training

### Teacher Network Training

In [ ]:
def standard_train(model, loader, num_epoch, optimizer, criterion, T=None, 
                   device=device):
  ###################### Problem 2 (4 points) ##################################
  # todo: Iterate over loader in each epoch                                    #
  # todo: Compute the model's output for each batch at the given temperature T #
  # todo: Compute the loss function and take a step by the optimizer           #
  # todo: Monitor the training procedure                                       #
  ##############################################################################

  # your code goes here
    
  ##############################################################################

In [ ]:
T = 100
teacher = Teacher(len(classes), T=T).to(device)
teacher_optim = optim.Adam(teacher.parameters())
teacher_criterion = nn.CrossEntropyLoss()

In [ ]:
standard_train(model=teacher, 
            loader=trainloader, 
            num_epoch=15, 
            optimizer=teacher_optim, 
            criterion=teacher_criterion, 
            T=T, 
            device=device)

### Student Network Training

In [ ]:
def distillation(teacher, student, loader, num_epoch, optimizer, criterion, 
                 T=None, device=device):
  ###################### Problem 3 (6 points) ##################################
  # todo: Iterate over loader in each epoch                                    #
  # todo: Compute MSE loss between student's logit and teacher's logit         #
  # todo: Take a step by the optimizer                                         #
  # todo: Monitor the training procedure                                       #
  ##############################################################################

  # your code goes here    

  ################################ End #########################################

In [ ]:
T = 100
student = Student(len(classes), T=T).to(device)
student_optim = optim.Adam(student.parameters())
std_criterion = nn.MSELoss()

In [ ]:
distillation(teacher=teacher, 
             student=student, 
             loader=trainloader, 
             num_epoch=100, 
             optimizer=student_optim, 
             criterion=std_criterion, 
             T=T, 
             device=device)

### Computing Clean Accuracy

In [ ]:
def standard_test(model, loader, T=1, device=device):
  correct = 0
  total = 0
  ###################### Problem 4 (3 points) ##################################
  # todo: Iterate over loader, compute the output and predicted                #
  # label, and update "correct" and "total" counters accordingly.              # 
  ##############################################################################
  
  # your code goes here
  
  ################################ End #########################################
  print(f'Clean accuracy of the network on the 10000 test images: {100 * correct // total} %')

In [ ]:
standard_test(student, testloader)

## Attacks

In [ ]:
def fgsm_attack (model, X, y, epsilon, T=1, mode="output"):
  delta = torch.zeros_like(X, requires_grad=True).to(device)
  ###################### Problem 5 (9 points) ######################
  # todo: Perform forward path on model with the image             #
  # todo: Compute loss:                                            #
  #       - In output mode, set cross entropy as the loss function #
  #       - In logit mode, set the logit value of the target label #
  #         as the loss function                                   #
  # todo: Perform backward path on loss function                   #
  # todo: Calculate the gradient w.r.t. the data                   #
  # todo: Determine delta based on the gradient and epsilon        #
  # Also, if the perturbed image exceeds the valid range, clamp    #
  # the delta in order to obtain an image in the valid range       #  
  ##################################################################

  # your code goes here

  ########################### End ##################################
    
  return delta + X

In [ ]:
def attack_test(model, attack_model, loader, mode="output", epsilon=4/255, T=1, 
                device=device):
  correct = 0
  total = 0
  ###################### Problem 6 (4.5 points) ################################
  # todo: Iterate over loader                                                  #
  # todo: Find an adversarial example by FGSM attack on attack_model           #
  # todo: Compute the output and predicted label, and updated "correct" and    #
  # "total" counters accordingly.                                              # 
  ##############################################################################

  # your code goes here
  
  ################################ End #########################################
  print(f'Accuracy of the network on the 10000 test images after attacking {mode} layer: {100 * correct // total} %')

In [ ]:
###################### Problem 7 (1.5 points) ##################################
# todo: Report the accuracy of the student model under attack output and logit #
#       layers                                                                 #
# todo: Do not forget to set the temperature of the student model to 1         #
################################################################################

# your code goes here

################################ End ###########################################